In [1]:
%load_ext jupyter_black

In [2]:
import pandas as pd
import re
from functools import partial
import pickle

In [3]:
students_raw = pd.read_csv("../data/students_dropout_raw.csv", sep = ";")

In [4]:
students_raw.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,8,5,2,1,1,1,13,10,6,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,6,1,11,1,1,1,1,3,4,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,5,1,1,1,22,27,10,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,8,2,15,1,1,1,23,27,6,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,12,1,3,0,1,1,22,28,10,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [5]:
def names_cleaning_1(df):
    df = df.copy()
    # Rename the columns
    df = df.rename(columns = lambda x: x.replace(" ","_").lower())
    # Rename the nationality column, spelled 'nacionality' in the original data
    df = df.rename(columns = {"nacionality":"nationality"})
    return df

In [6]:
students = students_raw.pipe(names_cleaning_1)

In [7]:
# No missing values in the data
students.isnull().sum()

marital_status                                    0
application_mode                                  0
application_order                                 0
course                                            0
daytime/evening_attendance                        0
previous_qualification                            0
nationality                                       0
mother's_qualification                            0
father's_qualification                            0
mother's_occupation                               0
father's_occupation                               0
displaced                                         0
educational_special_needs                         0
debtor                                            0
tuition_fees_up_to_date                           0
gender                                            0
scholarship_holder                                0
age_at_enrollment                                 0
international                                     0
curricular_u

In [8]:
# All columns are numeric
students.dtypes

marital_status                                      int64
application_mode                                    int64
application_order                                   int64
course                                              int64
daytime/evening_attendance                          int64
previous_qualification                              int64
nationality                                         int64
mother's_qualification                              int64
father's_qualification                              int64
mother's_occupation                                 int64
father's_occupation                                 int64
displaced                                           int64
educational_special_needs                           int64
debtor                                              int64
tuition_fees_up_to_date                             int64
gender                                              int64
scholarship_holder                                  int64
age_at_enrollm

In [9]:
students.describe().T

,count,mean,std,min,25%,50%,75%,max
marital_status,4424.0,1.178571,0.605747,1.00,1.00,1.000000,1.000000,6.000000
application_mode,4424.0,6.886980,5.298964,1.00,1.00,8.000000,12.000000,18.000000
application_order,4424.0,1.727848,1.313793,0.00,1.00,1.000000,2.000000,9.000000
course,4424.0,9.899186,4.331792,1.00,6.00,10.000000,13.000000,17.000000
daytime/evening_attendance,4424.0,0.890823,0.311897,0.00,1.00,1.000000,1.000000,1.000000
previous_qualification,4424.0,2.531420,3.963707,1.00,1.00,1.000000,1.000000,17.000000
nationality,4424.0,1.254521,1.748447,1.00,1.00,1.000000,1.000000,21.000000
mother's_qualification,4424.0,12.322107,9.026251,1.00,2.00,13.000000,22.000000,29.000000
father's_qualification,4424.0,16.455244,11.044800,1.00,3.00,14.000000,27.000000,34.000000
mother's_occupation,4424.0,7.317812,3.997828,1.00,5.00,6.000000,10.000000,32.000000


First interesting points in the data:
*   The majority of the students are single

Now let's match the categorical columns with the original categories, loading the dictionary scraped from the site

In [10]:
with open("../data/categorical_maps.pkl", "rb") as f:
    data_map = pickle.load(f)

In [11]:
cat_columns = [k for k,v in data_map.items() if not isinstance(v,str)]
num_columns_dict = {k:v for k,v in data_map.items() if isinstance(v,str)}

In [12]:
cat_columns

['marital_status',
 'nationality',
 'displaced',
 'gender',
 'international',
 "father's_qualification",
 "mother's_qualification",
 "father's_occupation",
 "mother's_occupation",
 'educational_special_needs',
 'debtor',
 'tuition_fees_up_to_date',
 'scholarship_holder',
 'application_mode',
 'course',
 'daytime/evening_attendance',
 'previous_qualification']

In [13]:
num_columns_dict

{'age_at_enrollment': 'Age of students at enrollment',
 'unemployment_rate': 'Unemployment rate (%)',
 'inflation_rate': 'Inflation rate (%)',
 'gdp': 'Gross domestic product',
 'application_order': 'Application order (between 0 - first choice; and 9 last choice)',
 'curricular_units_1st_sem_(credited)': 'Number of curricular units credited in the 1st semester',
 'curricular_units_1st_sem_(enrolled)': 'Number of curricular units enrolled in the 1st semester',
 'curricular_units_1st_sem_(evaluations)': 'Number of evaluations to curricular units in the 1st semester',
 'curricular_units_1st_sem_(approved)': 'Number of curricular units approved in the 1st semester',
 'curricular_units_1st_sem_(grade)': 'Grade average in the 1st semester (between 0 and 20)',
 'curricular_units_1st_sem_(without_evaluations)': 'Number of curricular units without evaluations in the 1st semester',
 'curricular_units_2nd_sem_(credited)': 'Number of curricular units credited in the 2nd semester',
 'curricular_uni

In [14]:
set(cat_columns).issubset(set(students.columns))

True

This mean that all the columns in our categorical columns dictionary are present in the data, that's a good sign. Now I'm going to replace the numerical values for the original categories of the data

In [15]:
def map_categories(df, cat_columns, mapping_dict):
    df = df.copy()
    for col in cat_columns:
        df[col] = df[col].map(mapping_dict.get(col))
    return df

In [16]:
students_cat = students.pipe(map_categories, cat_columns, data_map)

Now that we have our original categories back, let's perform a further clean of the column names

In [17]:
def names_cleaning_2(df):
    df = df.copy()
    df = df.rename(columns = {"daytime/evening_attendance":"day/evening",
                              "educational_special_needs":"special_needs",
                              "gdp":"gdp_change"})
    df = df.rename(columns = lambda x: x.replace("'s","").replace("curricular_units_","").replace("1st","first").replace("2nd","second").replace("qualification","education"))
    df = df.rename(columns = lambda x: re.sub(r"\((.*)\)",r"\1",x))
    return df

In [18]:
students_cat = students_cat.pipe(names_cleaning_2)

In [19]:
# Number of categories in the parents occupation and qualification columns

print(f'The father occupation column have {students_cat["father_occupation"].nunique()} distinct categories')
print(f'The mother occupation column have {students_cat["mother_occupation"].nunique()} distinct categories')
print(f'The father education column have {students_cat["father_education"].nunique()} distinct categories')
print(f'The mother education column have {students_cat["mother_education"].nunique()} distinct categories')

The father occupation column have 46 distinct categories
The mother occupation column have 32 distinct categories
The father education column have 34 distinct categories
The mother education column have 29 distinct categories


These are too many different values either for data visualization or machine learning. I will lump some categories together through dictionaries. Also, I will order to the education column categories based on years of education

In [20]:
education_map = {
    "Higher Education": "Higher Education",
    "Secondary Education|12th": "Secondary Education",
    "Basic education 1st cycle|4th|5th": "Basic education 1st cycle",
    "Basic Education 3rd Cycle|9th|10th|11th": "Basic Education 3rd Cycle",
    "Basic Education 2nd Cycle|6th|7th|8th": "Basic Education 2nd Cycle",
    "Unknown|Can't read": "Unknown",
    "2nd cycle of the general high school": "Basic Education 3rd Cycle",
    "course|Supplementary": "Tertiary course",
}

education_order = [
    "Unknown",
    "Basic education 1st cycle",
    "Basic Education 2nd Cycle",
    "Basic Education 3rd Cycle",
    "Secondary Education",
    "Tertiary course",
    "Higher Education",
]

occupation_map = {
    "Unskilled": "Unskilled",
    "Administrative": "Administrative",
    "Skilled": "Skilled",
    "Technicians": "Technicians",
    "Specialists": "Specialists",
    "Operators": "Operators",
    "Personal Services, Security and Safety Workers and Sellers": "Personal Services, Security and Safety Workers and Sellers",
    "Managers": "Managers",
    "Armed Forces Professions":"Armed Forces",
    "Student":"Student",    
}

In [21]:
def map_values(x, map_dict):
    try:
        return next(
            map_dict[k] for k in map_dict if re.search(k, x, flags=re.IGNORECASE)
        )
    except:
        return "Other"


map_education = partial(map_values, map_dict=education_map)

map_occupation = partial(map_values, map_dict=occupation_map)

In [22]:
students_cat["father_occupation"] = students_cat["father_occupation"].map(map_occupation)
students_cat["mother_occupation"] = students_cat["mother_occupation"].map(map_occupation)
students_cat["father_education"] = students_cat["father_education"].map(map_education)
students_cat["mother_education"] = students_cat["mother_education"].map(map_education)

In [23]:
#Check the new number of distinct categories

print(f'The father occupation column have {students_cat["father_occupation"].nunique()} distinct categories')
print(f'The mother occupation column have {students_cat["mother_occupation"].nunique()} distinct categories')
print(f'The father education column have {students_cat["father_education"].nunique()} distinct categories')
print(f'The mother education column have {students_cat["mother_education"].nunique()} distinct categories')

The father occupation column have 11 distinct categories
The mother occupation column have 11 distinct categories
The father education column have 7 distinct categories
The mother education column have 7 distinct categories


In [24]:
# Order the education columns
students_cat["father_education"] = students_cat["father_education"].astype("category").cat.reorder_categories(education_order)
students_cat["mother_education"] = students_cat["mother_education"].astype("category").cat.reorder_categories(education_order)

In [25]:
# Order the target column
target_order = ["Dropout", "Enrolled", "Graduate"]
students_cat["target"] = (
    students_cat["target"].astype("category").cat.reorder_categories(target_order)
)

In [27]:
# Save the processed file to parquet, this keeps the format in the columns (ie: keeps the order created for some categorical columns)
students_cat.to_parquet("../data/students_processed_cats.parquet", index = False)